In [1]:
import pandas as pd
import numpy as np

In [2]:
movies = pd.read_csv("./datasets/movies.csv")
ratings = pd.read_csv("./datasets/ratings.csv")
tags = pd.read_csv("./datasets/tags.csv")

In [3]:
df = pd.merge(ratings, movies, on = "movieId", how = "left")
df = df.drop('title', axis = 1)
df.head()

,userId,movieId,rating,timestamp,genres
0,1,1,4.0,964982703,Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Comedy|Romance
2,1,6,4.0,964982224,Action|Crime|Thriller
3,1,47,5.0,964983815,Mystery|Thriller
4,1,50,5.0,964982931,Crime|Mystery|Thriller


In [4]:
df['genres'] = df['genres'].str.split("|")

In [5]:
df.head()

,userId,movieId,rating,timestamp,genres
0,1,1,4.0,964982703,"[Adventure, Animation, Children, Comedy, Fantasy]"
1,1,3,4.0,964981247,"[Comedy, Romance]"
2,1,6,4.0,964982224,"[Action, Crime, Thriller]"
3,1,47,5.0,964983815,"[Mystery, Thriller]"
4,1,50,5.0,964982931,"[Crime, Mystery, Thriller]"


In [6]:
tags['tag'] = tags['tag'].str.split('|')
tags.drop('timestamp', axis=1, inplace=True)

In [7]:
tags = tags.groupby(['userId','movieId'])['tag'].apply(lambda x: ','.join(x.astype(str))).reset_index()
tags.head()

,userId,movieId,tag
0,2,60756,"['funny'],['Highly quotable'],['will ferrell']"
1,2,89774,"['Boxing story'],['MMA'],['Tom Hardy']"
2,2,106782,"['drugs'],['Leonardo DiCaprio'],['Martin Scors..."
3,7,48516,['way too long']
4,18,431,"['Al Pacino'],['gangster'],['mafia']"


In [8]:
df = pd.merge(df, tags, on=['userId','movieId'], how='left')

In [9]:
df.shape

(100836, 6)

In [10]:
df['tag'] = df['tag'].apply(lambda d: d if isinstance(d, list) else [])
df['genres'] = df['genres'].apply(lambda d: d if isinstance(d, list) else [])

In [11]:
df.head()

,userId,movieId,rating,timestamp,genres,tag
0,1,1,4.0,964982703,"[Adventure, Animation, Children, Comedy, Fantasy]",[]
1,1,3,4.0,964981247,"[Comedy, Romance]",[]
2,1,6,4.0,964982224,"[Action, Crime, Thriller]",[]
3,1,47,5.0,964983815,"[Mystery, Thriller]",[]
4,1,50,5.0,964982931,"[Crime, Mystery, Thriller]",[]


In [14]:
#boyut azaltma

min_ratings = 5
filter_items = df['movieId'].value_counts() > min_ratings
filter_items = filter_items[filter_items].index.tolist()

min_user_ratings = 5
filter_users = df['userId'].value_counts() > min_user_ratings
filter_users = filter_users[filter_users].index.tolist()

df_new = df[(df['movieId'].isin(filter_items)) & (df['userId'].isin(filter_users))]

print('Original shape {}'.format(df.shape))
print('New shape {}'.format(df_new.shape))

Original shape (100836, 6)
New shape (88364, 6)


In [16]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(df_new, test_size=0.2, stratify=df_new.userId)

In [17]:
train_data = train_data.sort_values(['userId', 'movieId'])
train_data.tail()

,userId,movieId,rating,timestamp,genres,tag
100829,610,164179,5.0,1493845631,[Sci-Fi],[]
100830,610,166528,4.0,1493879365,"[Action, Adventure, Fantasy, Sci-Fi]",[]
100831,610,166534,4.0,1493848402,"[Drama, Horror, Thriller]",[]
100832,610,168248,5.0,1493850091,"[Action, Crime, Thriller]",[]
100834,610,168252,5.0,1493846352,"[Action, Sci-Fi]",[]


In [18]:
test_data = test_data.sort_values(['userId','movieId'])
test_data.tail()

,userId,movieId,rating,timestamp,genres,tag
100782,610,139644,4.5,1493846253,"[Crime, Drama, Mystery]",[]
100794,610,143385,4.0,1493849643,"[Drama, Thriller]",[]
100807,610,152077,4.0,1493845817,[Thriller],[]
100816,610,158872,3.5,1493848024,"[Animation, Comedy]",[]
100833,610,168250,5.0,1494273047,[Horror],[]


In [19]:
train_data.to_csv('training_data.csv', index = False)
test_data.to_csv('testing_data.csv', index = False)

In [20]:
movies['genres'] = movies['genres'].str.split('|')
movies['genres'] = movies['genres'].apply(lambda d: d if isinstance(d, list) else [])
movies.head()
movies.to_csv('movies.csv', index = False)